<a href="https://colab.research.google.com/github/pyoph/ProjetHDP/blob/main/projetHDP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import cvxpy as cp
import numpy as np

In [62]:
#Premiers essais relaxation convexe (p,q) SBM : X_i,i = 1


np.random.seed(1)

#Paramètres
n = 3
p = np.random.uniform(0,1)
q = np.random.uniform(0,1)

J3 = np.array([[1,1,1],[1,1,1],[1,1,1]])

#Tuning parameter
l = (np.log(p/(1 - p)) - np.log(q/(1 - q)))/(np.log(p) - np.log(q) + np.log(1 - q) - np.log(1 - p))

#Définition de la matrice C observée
C = np.random.uniform(0, 1,size = (n,n))


C[0][0] = 0
C[1][1] = 0
C[2][2] = 0

print(C)

A = []
b = []
for i in range(n):
    b.append(1)

    A.append(np.array([[1,0,0], [0,0,0], [0,0,0]]))
    A.append(np.array([[0,0,0], [0,1,0], [0,0,0]]))
    A.append(np.array([[0,0,0], [0,0,0], [0,0,1]]))



# Define and solve the CVXPY problem.
# Create a symmetric matrix variable.
X = cp.Variable((n,n), symmetric=True)
# The operator >> denotes matrix inequality.
constraints = [X >> 0]
constraints += [
    cp.trace(A[i] @ X) == b[i] for i in range(n)
]
prob = cp.Problem(cp.Minimize(cp.trace((C - l*J3) @ X)),
                  constraints)
prob.solve()

# Print result.
print("The optimal value is", prob.value)
print("A solution X is")
print(X.value)

[[0.         0.30233257 0.14675589]
 [0.09233859 0.         0.34556073]
 [0.39676747 0.53881673 0.        ]]
The optimal value is -7.177427848075446
A solution X is
[[1.         0.999999   0.99999862]
 [0.999999   1.         1.0000031 ]
 [0.99999862 1.0000031  1.        ]]
